# Slash Blur

John Mays | Created: 07/05/2025 | Last Updated: 07/05/2025

## Setup:

In [17]:
from PIL import Image
import random

In [30]:
# global variables
concentration = 0.5 # average slashes per pixel
slash_length = 5 # unit: pixels
slash_intensity = 1 # balance between slash and original pixels | 0 is all original. 1 is all slash.
image_bounds = None

## Import:

In [2]:
e = Image.open('../data/in/test_e_big.png')

In [5]:
e.size

(1536, 864)

In [7]:
e.putpixel((10, 10), (255, 0, 0))

In [8]:
e.getpixel((10,10))


(255, 0, 0)

In [31]:
image_bounds = (0, 0) + e.size

In [33]:
image_bounds

(0, 0, 1536, 864)

## Algorithm:

In [34]:
class Blur:
    """
    One instance of the Blur class represents one parameterized instance
    of the blur algorithm.
    """

    def Blur(self, concentration, slash_length, slash_intensity):
        self.concentration = concentration # average slashes per pixel
        self.slash_length = slash_length # unit: pixels
        self.slash_intensity = slash_intensity # balance between slash and original pixels | 0 is all original. 1 is all slash.
        self.image_bounds = None
    
    def within(self, pixel:tuple) -> bool:
        x, y = pixel
        if x >= self.image_bounds[0] and x < self.image_bounds[2]\
        and y >= self.image_bounds[1] and y < self.image_bounds[3]:
            return True
        return False

    def perform_blur(self, im:Image.Image) -> Image.Image:
        self.image_bounds = (0, 0) + im.size
        pass
        return im

In [ ]:
def within

In [ ]:
def get_slash_pixels(center_pixel, slope):
    slash_pixels = [center_pixel]
    

    slash_pixels = [p for p in slash_pixels if ]

In [ ]:
def perform_slash(im, center_pixel: tuple) -> None:
    slash_slope = random.uniform(-1,1)
    

In [ ]:
def perform_blur(image: Image.Image) -> Image.Image: